# Equal-Weight S&P 500 Index Fund Creation

This project automates the creation of an equal-weight S&P 500 index fund. We'll import necessary libraries, fetch stock data, and calculate the distribution of funds across all S&P 500 stocks.

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Stock Data Import

We begin by importing the list of S&P 500 stocks. The dataset is available as a CSV file.

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')

## API Token and Data Retrieval

To access real-time stock data, we use the IEX Cloud API. The API token is stored separately for security.

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

## Fetching Stock Data

Using the IEX Cloud API, we retrieve key data for each stock: its market capitalization and current price.

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Data Parsing and DataFrame Setup

Parse the retrieved data and set up a DataFrame to hold our stock information.

In [ ]:
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

## Batch API Calls

To enhance performance, we use batch API calls, allowing us to fetch data for multiple stocks in a single request.

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = [','.join(group) for group in symbol_groups]
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['marketCap'], 'N/A'],
                      index = my_columns),
            ignore_index = True)

## Portfolio Allocation

Determine the number of shares to buy for each stock based on an equal-weight strategy.

In [ ]:
portfolio_size = float(input("Enter the value of your portfolio:"))
position_size = portfolio_size / len(final_dataframe.index)
final_dataframe['Number Of Shares to Buy'] = final_dataframe['Price'].apply(lambda x: math.floor(position_size / x))

## Excel Output

Generating an Excel file with the recommended trades, using custom formatting for better readability.

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)
# Apply formatting...
# Save Excel file
writer.save()